# Sequential ensembles: Newton boosting

Al igual que con la potenciación de gradiente, el marco de la potenciación de Newton se puede aplicar a cualquier función de pérdida, lo que significa que cualquier problema de clasificación o regresión se puede potenciar utilizando aprendices débiles. Además de esta flexibilidad, ahora están disponibles paquetes como XGBoost que pueden escalar el impulso de Newton a big data a través de la paralelización. Como era de esperar, muchos profesionales consideran actualmente que el impulso de Newton es un enfoque de conjunto de última generación.

Los orígenes y la motivación para diseñar el impulso de Newton son análogos a los del algoritmo de aumento de gradiente: la optimización de las funciones de pérdida. El descenso de gradiente, en el que se basa el aumento de gradiente, es un método de optimización de primer orden en el que utiliza primeras derivadas durante la optimización. 

El método de Newton, o el descenso de Newton, es un método de optimización de segundo orden, ya que utiliza información de la primera y la segunda derivada juntas para calcular un paso de Newton. Cuando se combina con refuerzo, obtenemos el método de conjunto de refuerzo de Newton.

**<center>Newton’s method for minimization</center>**

<img src="img/newton.jpg">

Los métodos de optimización iterativos, como el descenso de gradiente y el método de Newton, realizan una actualización dentro de cada iteración: siguiente = actual + (paso × dirección). En el descenso de gradiente (figura izquierda), la información de la primera derivada solo nos permite construir una aproximación lineal local en el mejor de los casos. Si bien esto nos da una dirección de descenso, diferentes longitudes de paso pueden darnos estimaciones muy diferentes y, en última instancia, pueden ralentizar la convergencia.

¡Incorporar información de la segunda derivada, como lo hace el descenso de Newton, nos permite construir una aproximación cuadrática local! Esta información adicional conduce a una mejor aproximación local, lo que resulta en mejores pasos y una convergencia más rápida.

* **Nota:** El método de optimización de Newton aqui utilizado, se deriva de un método más general de búsqueda de raíces, también llamado método de Newton.

* **Nota:** A diferencia del descenso de gradiente, el descenso de Newton calcula pasos exactos y no requiere un cálculo de longitud de paso. Sin embargo, incluiremos explícitamente la longitud del paso por dos razones: (1) para permitirnos comparar y comprender de inmediato las diferencias entre el descenso de gradiente y el descenso de Newton; y (2) lo que es más importante, a diferencia del descenso de Newton, el impulso de Newton solo puede aproximar el paso y requerirá que especifiquemos una longitud de paso similar al descenso de gradiente y al aumento de gradiente. Como veremos, esta longitud de paso en el impulso de Newton no es más que la tasa de aprendizaje.

### Newton’s descent

In [ ]:
import numpy as np


# Newton’s descent requires a function f, its gradient g, and its Hessian h.
def newton_descent(f, g, h, x_init, max_iter=100, args=()):
    converged = False # Initializes Newton’s descent to not converged
    n_iter = 0
    x_old, x_new = np.array(x_init), None
    descent_path = np.full((max_iter + 1, 2), fill_value=np.nan)
    descent_path[n_iter] = x_old
    while not converged:
        n_iter += 1
        gradient = g(x_old, *args) # Computes the gradient /
        hessian = h(x_old, *args)  # and the Hessian
        direction = -np.dot(np.linalg.inv(hessian), gradient) # Computes the Newton direction
        distance = 1 # Sets step length to 1, for simplicity
        x_new = x_old + distance * direction # Computes the update
        descent_path[n_iter] = x_new
        err = np.linalg.norm(x_new - x_old) # Computes change from previous iteration
        if err <= 1e-3 or n_iter >= max_iter:
            converged = True # Converges if change is small or maximum iterations are reached
        x_old = x_new # Gets ready for the next iteration
    return x_new, descent_path

<center>Newton’s descent compare to gradient descent (Example)</center>

<img src="img/newton_descent.jpg">

Notamos un par de cosas importantes sobre el descenso de Newton y sus similitudes con el descenso de gradiente. 
* Primero, a diferencia del descenso de gradiente, el método de Newton calcula exactamente el paso de descenso y no requiere una longitud de paso. Tenga en cuenta que nuestro propósito es extender el descenso de Newton al impulso de Newton. Desde esta perspectiva, la longitud del paso puede interpretarse como una tasa de aprendizaje. Elegir una tasa de aprendizaje efectiva (por ejemplo, usar la validación cruzada como hicimos con AdaBoost o el aumento de gradiente) es muy parecido a elegir una buena longitud de paso. En lugar de seleccionar una tasa de aprendizaje para acelerar la convergencia, al impulsar los algoritmos, seleccionamos la tasa de aprendizaje para ayudarnos a evitar el sobreajuste y generalizar mejor el conjunto de prueba y los datos futuros.
* Un segundo punto importante a tener en cuenta es que, al igual que el descenso de gradiente, la función de Newton
el descenso también es sensible a nuestra elección del punto inicial. Diferentes inicializaciones llevarán el descenso de Newton a diferentes minimizadores locales. Además de los minimizadores locales, un problema mayor es que nuestra elección del punto inicial también puede hacer que el descenso de Newton converja en puntos de silla. Este es un problema al que se enfrentan todos los algoritmos de descenso. Un punto de silla imita a un minimizador local: en ambas ubicaciones, el gradiente de la función se vuelve cero. Sin embargo, los puntos de silla no son verdaderos minimizadores locales: la forma de silla de montar significa que se curva hacia arriba en una dirección y hacia abajo en otra. Esto contrasta con los minimizadores locales, que tienen forma de cuenco. Sin embargo, tanto los minimizadores locales como los puntos de silla tienen gradientes cero. Esto significa que los algoritmos de descenso no pueden distinguir entre los dos y, a veces, convergen en puntos de silla en lugar de minimizadores.

### Newton’s descent over loss functions for training (EXAMPLE)

In [ ]:
from sklearn.datasets import make_blobs


X, y = make_blobs(n_samples=200, n_features=2, centers=[[-1.5, -1.5], [1.5, 1.5]])

def squared_loss(w, X, y):
    return 0.5 * np.sum((y - np.dot(X, w))**2)

def squared_loss_gradient(w, X, y):
    return -np.dot(X.T, (y - np.dot(X, w)))

def squared_loss_hessian(w, X, y):
    return np.dot(X.T, X)

w_init = np.array([0.0, -0.99])

# Implemented in Sequential_Ensembles_Gradient_Boosting
w_gradient, path_gradient = gradient_descent(
    squared_loss, 
    squared_loss_gradient, 
    w_init, 
    args=(X, y)
    )

w_newton, path_newton = newton_descent(
    squared_loss, 
    squared_loss_gradient, 
    squared_loss_hessian, 
    w_init, 
    args=(X, y))

print(w_gradient)
print(w_newton)

ypred = (np.dot(X, w_newton) >= 0).astype(int)

from sklearn.metrics import accuracy_score

accuracy_score(y, ypred)

<img src="img/newton_example.jpg">

## Newton boosting: Newton’s method + boosting

Al igual que con otros métodos de impulso, el impulso de Newton aprende un nuevo estimador débil en cada iteración, de modo que corrige las clasificaciones erróneas o los errores cometidos por la iteración anterior. 

* AdaBoost identifica y caracteriza los ejemplos mal clasificados que necesitan atención asignándoles pesos: a los ejemplos mal clasificados se les asignan pesos más altos. Un clasificador débil entrenado en tales ejemplos ponderados se centrará más en ellos durante el aprendizaje.
* El aumento de gradiente caracteriza los ejemplos mal clasificados que necesitan atención a través de residuos. Un residual es simplemente otro medio para medir el grado de clasificación errónea y se calcula como el gradiente de la función de pérdida.

¡El impulso de Newton hace ambas cosas y usa residuos ponderados! Los residuos en el aumento de Newton se calculan exactamente de la misma manera que en el aumento de gradiente: utilizando el gradiente de la función de pérdida (la primera derivada). Los pesos, por otro lado, se calculan utilizando la Hessian de la función de pérdida (la segunda derivada).

### Learning with regularized loss functions

Una función de pérdida regularizada contiene un término de suavizado adicional junto con la función de pérdida, lo que la hace más convexa o similar a un cuenco. La regularización de una función de pérdida introduce una estructura adicional al problema de aprendizaje, que a menudo estabiliza y acelera los algoritmos de aprendizaje resultantes. La regularización también nos permite controlar la complejidad del modelo que se está aprendiendo y mejora la solidez general y las capacidades de generalización del modelo. Esencialmente, una función de pérdida regularizada captura explícitamente el equilibrio entre ajuste y complejidad inherente a la mayoría de los modelos de aprendizaje automático.

El término de regularización mide la planitud (lo opuesto a la "curvatura") del modelo: cuanto más se minimiza, menos complejo es el modelo aprendido.

El término de pérdida mide el ajuste a los datos de entrenamiento a través de una función de pérdida: cuanto más se minimice, mejor será el ajuste a los datos de entrenamiento.

El parámetro de regularización se compensa entre estos dos objetivos en competencia:
* Un valor alto de $\lambda$ significa que el modelo se centrará más en la regularización y la simplicidad y menos en el error de entrenamiento, lo que hace que el modelo tenga un mayor error de entrenamiento y ajuste insuficiente.
* Un valor pequeño de $\lambda$ significa que el modelo se centrará más en los errores de entrenamiento y aprenderá modelos más complejos, lo que hace que el modelo tenga menos errores de entrenamiento y posiblemente se sobreajuste. 

Por lo tanto, una función de pérdida regularizada nos permite equilibrar el ajuste y la complejidad durante el aprendizaje, lo que en última instancia conduce a modelos que se generalizan bien en la práctica.

### Regularized logistic loss, gradient, and Hessian with $\lambda$ = 0.1

In [ ]:
def log_loss_func(y, F):
    return np.log(1 + np.exp(-y * F)) + 0.1 * F**2

def log_loss_grad(y, F):
    return -y / (1 + np.exp(y * F)) + 0.2 * F

def log_loss_hess(y, F):
    return np.exp(y * F) / (1 + np.exp(y * F))**2 + 0.2

Estas funciones ahora se pueden usar para calcular los residuos y los pesos hessianos correspondientes que necesitaremos para el refuerzo de Newton.

### Newton boosting for the regularized logistic loss

In [ ]:
from sklearn.tree import DecisionTreeRegressor
from scipy.optimize import minimize_scalar


def fit_gradient_boosting(X, y, n_estimators=10, use_newton=True):
    n_samples, n_features = X.shape # Gets dimensions of the data set
    estimators = [] # Initializes an empty ensemble
    F = np.full((n_samples, ), 0.0) # Predictions of the ensemble on the training set
    for t in range(n_estimators):
        if use_newton: # If Newton boosting, computes Hessian-weighted residuals
            residuals = -log_loss_grad(y, F) / log_loss_hess(y, F)
        else:
            residuals = -log_loss_grad(y, F) # Else computes unweighted residuals for gradient boosting
        h = DecisionTreeRegressor(max_depth=1)
        h.fit(X, residuals) # Fits weak regression tree (ht) to the examples and residuals
        hreg = h.predict(X) # Gets predictions of the weak learner, ht
        loss = lambda a: np.linalg.norm(y - (F + a * hreg))**2 # Sets up the loss function as a line search problem
        step = minimize_scalar(loss, method='golden') # Finds the best step length using the golden section search
        a = step.x
        F += a * hreg # Updates the ensemble predictions
        estimators.append((a, h)) # Updates the ensemble
    return estimators

Una vez que se aprende el modelo, podemos hacer predicciones exactamente como con AdaBoost o el aumento de gradiente porque el conjunto aprendido es un conjunto secuencial. La siguiente lista es la misma función de predicción utilizada por estos métodos presentados anteriormente, que se repite aquí por conveniencia.

In [ ]:
def predict_gradient_boosting(X, estimators):
    pred = np.zeros((X.shape[0], )) # Initializes all the predictions to 0
    for a, h in estimators:
        pred += a * h.predict(X) # Aggregates individual predictions from each regressor
    y = np.sign(pred) # Converts weighted predictions to –1/1 labels

    return y

Comparemos el rendimiento de nuestras implementaciones de refuerzo de gradiente y refuerzo de Newton:

In [ ]:
# EXAMPLE
from sklearn.datasets import make_moons


X, y = make_moons(n_samples=200, noise=0.15, random_state=13)
y = 2 * y - 1 # Converts training labels to –1/1

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Splits into train and test sets
Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.25, random_state=11)

# Newton boosting
estimators_nb = fit_gradient_boosting(Xtrn, ytrn, n_estimators=25, use_newton=True)

ypred_nb = predict_gradient_boosting(Xtst, estimators_nb)

print('Newton boosting test error = {0}'.format(1 - accuracy_score(ypred_nb, ytst)))

# Gradient boosting
estimators_gb = fit_gradient_boosting(Xtrn, ytrn, n_estimators=25, use_newton=False)

ypred_gb = predict_gradient_boosting(Xtst, estimators_gb)

print('Gradient boosting test error = {0}'.format(1 - accuracy_score(ypred_gb, ytst)))

## XGBoost: A framework for Newton boosting

XGBoost, o eXtreme Gradient Boosting, es un marco de código abierto para potenciar gradientes (originado a partir de un proyecto de investigación de Tianqi Chen). Obtuvo un amplio reconocimiento y adopción, especialmente en la comunidad de competencia de ciencia de datos, después de su éxito en el Higgs Boson Machine Learning Challenge.
Desde entonces, XGBoost ha evolucionado hasta convertirse en un poderoso marco de impulso que proporciona capacidades de procesamiento distribuido y paralelización que le permiten escalar a conjuntos de datos muy grandes. Actualmente, XGBoost está disponible en muchos idiomas, incluidos Python, R y C/C++, y se implementa en varias plataformas de ciencia de datos, como Apache Spark y H2O. XGBoost tiene varias características clave que lo hacen aplicable en una variedad de dominios, así como para datos a gran escala:
* Impulso de Newton en funciones de pérdida regularizadas para controlar directamente la complejidad de las funciones del árbol de regresión (aprendices débiles) que constituyen el conjunto
* Aceleraciones algorítmicas como el boceto de cuantiles ponderados, una variante del algoritmo de búsqueda dividida basado en histogramas (que usa LightGBM) para un entrenamiento más rápido
* Compatibilidad con una gran cantidad de funciones de pérdida para clasificación, regresión y clasificación, así como funciones de pérdida personalizadas específicas de la aplicación, similares a LightGBM
* Diseño de sistema basado en bloques que almacena datos en la memoria en unidades más pequeñas llamadas bloques; esto permite un aprendizaje paralelo, un mejor almacenamiento en caché y subprocesos múltiples eficientes.

Es imposible detallar todas las funciones disponibles en XGBoost, esta sera una introduccion a XGBoost, su uso, y aplicaciones en entornos prácticos. Esto le permitirá avanzar más hacia los casos de uso avanzados de XGBoost para sus aplicaciones a través de su documentación.

### Implementing Newton boosting with XGBoost

In [ ]:
# We kick off our explorations of XGBoost with the breast cancer data set
from sklearn.datasets import load_breast_cancer
from sklearn.model_selection import train_test_split


X, y = load_breast_cancer(return_X_y=True)
Xtrn, Xtst, ytrn, ytst = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

from xgboost import XGBClassifier

# XGBoost provides a familiar interface that is designed to look and feel like scikit-learn
ens = XGBClassifier(
    n_estimators=20, 
    max_depth=1, 
    objective='binary:logistic')

ens.fit(Xtrn, ytrn)

# It’s also similarly easy to predict labels on test data and evaluate model performance:
from sklearn.metrics import accuracy_score

ypred = ens.predict(Xtst)
accuracy_score(ytst, ypred)

Como alternativa, podemos usar la interfaz nativa de XGBoost, que se diseñó originalmente para leer datos en el formato LIBSVM, que es muy adecuado para almacenar datos dispersos con muchos ceros de manera eficiente.
En el formato LIBSVM, cada línea del archivo de datos contiene un solo ejemplo de entrenamiento representado de la siguiente manera:

XGBoost usa un objeto de datos llamado DMatrix para agrupar los datos y las etiquetas correspondientes. Los objetos DMatrix se pueden crear leyendo datos directamente de archivos o de otros objetos similares a matrices. Aquí, creamos dos objetos DMatrix llamados trn y tst para representar las matrices de datos de prueba y entrenamiento:

In [ ]:
import xgboost as xgb

trn = xgb.DMatrix(Xtrn, label=ytrn)
tst = xgb.DMatrix(Xtst, label=ytst)

# We also set up the training parameters using a dictionary and train an XGBoost model using trn and the parameters:
params = {'max_depth': 1, 'objective':'binary:logistic'}
ens2 = xgb.train(params, trn, num_boost_round=20)

ypred_proba = ens2.predict(tst)
ypred = (ypred_proba >= 0.5).astype(int)

accuracy_score(ytst, ypred)

Se debe tener cuidado al usar este modelo para la predicción. Los modelos entrenados con ciertas funciones de pérdida devolverán probabilidades de predicción en lugar de las predicciones directamente. La función de pérdida logística es uno de esos casos. Estas probabilidades de predicción se pueden convertir en etiquetas de clasificación binaria 0/1 mediante un umbral de 0,5. Es decir, todos los ejemplos de prueba con probabilidad de predicción ≥ 0,5 se clasifican en Clase 1 y el resto en Clase 0:

Finalmente, XGBoost admite tres tipos diferentes de enfoques de refuerzo, que se pueden configurar a través del parámetro de refuerzo:
* booster='gbtree' es la configuración predeterminada e implementa el impulso de Newton usando árboles como alumnos débiles entrenados usando regresión basada en árboles.
* booster='gblinear' implementa el impulso de Newton usando funciones lineales como alumnos débiles entrenados usando regresión lineal.
* booster='dart' entrena un conjunto usando Dropouts Meet Multiple Additive Regression Trees (DART).

Al igual que con AdaBoost y el aumento de gradiente, buscamos establecer la tasa de aprendizaje o emplear la detención temprana como un medio para controlar el sobreajuste, de la siguiente manera:
* Al seleccionar una tasa de aprendizaje efectiva, tratamos de controlar la tasa a la que aprende el modelo para que no se ajuste rápidamente y luego se ajuste en exceso a los datos de entrenamiento. Podemos pensar en esto como un enfoque de modelado proactivo, donde tratamos de identificar una buena estrategia de capacitación para que conduzca a un buen modelo.
* Al imponer la detención temprana, intentamos detener el entrenamiento tan pronto como observamos que el modelo comienza a sobreajustarse. Podemos pensar en esto como un enfoque de modelado reactivo, donde contemplamos terminar el entrenamiento tan pronto como creamos que tenemos un buen modelo.

LEARNING RATE VIA CROSS VALIDATION

CV se puede usar tanto para establecer la tasa de aprendizaje como para seleccionar otros parámetros de aprendizaje, como ser la profundidad máxima del árbol, la cantidad de nodos de hoja e incluso parámetros específicos de la función de pérdida.

Combinamos la clase StratifiedKFold de scikit-learn para dividir los datos de entrenamiento en 10 pliegues de conjuntos de entrenamiento y validación. StratifiedKFold asegura que preservamos las distribuciones de clases, es decir, las fracciones de diferentes clases en los pliegues.

In [ ]:
# First, we initialize the learning rates we’re interested in exploring:
import numpy as np

learning_rates = np.concatenate([np.linspace(0.02, 0.1, num=5), np.linspace(0.2, 1.8, num=9)])
n_learning_rate_steps = len(learning_rates)
print(learning_rates)

# Next, we set up StratifiedKFold to split the training data into 10 folds:
from sklearn.model_selection import StratifiedKFold

n_folds = 10
splitter = StratifiedKFold(n_splits=n_folds, shuffle=True, random_state=42)

#### Cross validation with XGBoost and scikit-learn

In [ ]:
trn_err = np.zeros((n_learning_rate_steps, n_folds))
val_err = np.zeros((n_learning_rate_steps, n_folds)) # Saves training and validation errors

# Trains an XGBoost classifier for each fold with different learning rates
for i, rate in enumerate(learning_rates): 
    for j, (trn, val) in enumerate(splitter.split(X, y)):
        gbm = XGBClassifier(n_estimators=10, max_depth=1, learning_rate=rate, verbosity=0)
        gbm.fit(X[trn, :], y[trn])
        # Saves training and validation errors
        trn_err[i, j] = (1 - accuracy_score(y[trn], gbm.predict(X[trn, :]))) * 100
        val_err[i, j] = (1 - accuracy_score(y[val], gbm.predict(X[val, :]))) * 100

# Averages training and validation errors across folds
trn_err = np.mean(trn_err, axis=1)
val_err = np.mean(val_err, axis=1)

<div>
    <div style="float:left; width:20%;">
        A medida que disminuye la tasa de aprendizaje, el rendimiento de XGBoost se degrada a medida que el proceso de impulso se vuelve cada vez más conservador y exhibe un comportamiento inadecuado. A medida que aumenta la tasa de aprendizaje, el rendimiento de XGBoost, una vez más, se degrada a medida que el proceso de impulso se vuelve cada vez más agresivo y muestra un comportamiento de sobreajuste. El mejor valor entre nuestras opciones de parámetros parece ser learning_rate=1.2, que generalmente está en la región entre 1.0 y 1.5.
    </div>
    <div style="float:right; width:70%;">
        <img src="img/lr_result.jpg">
    </div>
</div>

#### Cross validation with XGBoost

CV también puede ser útil para caracterizar el rendimiento del modelo. Usaremos la funcionalidad CV integrada de XGBoost para caracterizar cómo cambia el rendimiento de XGBoost a medida que aumentamos la cantidad de estimadores en el conjunto.
Usamos la función XGBoost.cv para realizar un CV de 10 veces. Observe que xgb.cv se llama casi de la misma manera que xgb.fit.

In [ ]:
import xgboost as xgb


trn = xgb.DMatrix(Xtrn, label=ytrn)
tst = xgb.DMatrix(Xtst, label=ytst)

params = {'learning_rate': 0.25, 'max_depth': 2, 'objective': 'binary:logistic'}

cv_results = xgb.cv(params, trn, num_boost_round=60, nfold=10, metrics={'error'}, seed=42)

<img src="img/xgboost_cv.jpg">
El rendimiento del entrenamiento y la validación deja de mejorar significativamente alrededor de las 35 iteraciones. Esto sugiere que no se puede obtener una mejora significativa en el rendimiento al prolongar el entrenamiento más allá de este punto. Esto nos lleva, bastante claramente, a la noción de detención anticipada, que hemos encontrado antes tanto con AdaBoost como con el aumento de gradiente.

#### Early Stopping

La detención temprana con XGBoost funciona de manera bastante similar a LightGBM, donde especificamos un valor para el parámetro early_stopping_rounds. El rendimiento del conjunto se califica después de cada iteración en un conjunto de validación, que se separa del conjunto de entrenamiento con el fin de identificar un buen punto de parada inicial. Siempre que la puntuación general (por ejemplo, la precisión) mejore con respecto a las últimas rondas de parada temprana, XGBoost seguirá entrenando. Sin embargo, cuando la puntuación no mejora después de early_stopping_rounds, XGBoost finaliza el entrenamiento. Tenga en cuenta que train_test_split se usa para crear un conjunto de validación independiente que XGBoost usa para identificar un punto de parada temprano.

In [ ]:
from sklearn.model_selection import train_test_split


Xtrn, Xval, ytrn, yval = train_test_split(X, y, test_size=0.2, shuffle=True, random_state=42)

ens = XGBClassifier(n_estimators=50, max_depth=2, objective='binary:logistic')

ens.fit(Xtrn, ytrn, early_stopping_rounds=5, eval_set=[(Xval, yval)], eval_metric='auc')

Los tres parámetros clave para la detención temprana en el codigo anterior son el número de rondas de detención temprana y el conjunto de evaluación: early_stop_rounds=5 y eval_set=[(Xval, yval)], y la métrica de evaluación eval_metric='auc'.